In [9]:
from vnstock import *
import yfinance as yf
import datetime
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.linear_model import LinearRegression
import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

In [10]:
data_ticker = pd.read_excel("C:\\Users\\Chi Bao\\OneDrive\\Desktop\\Đồ án tốt nghiệp\\data_ticker.xlsx")

FI = ['GC=F','SI=F','CL=F','^GSPC','^DJI','^IXIC','^RUT','^FTSE','^N225','^NYA']
df = pd.DataFrame()
for symbol in FI:
    data = yf.download(symbol, start='2012-03-20', end='2023-09-06')
    df[symbol] = data['Adj Close']

column_mapping = {
    'GC=F': 'Gold',
    'SI=F': 'Silver',
    'CL=F': 'Crude Oil',
    '^GSPC': 'S&P 500',
    '^DJI': 'Dow Jones',
    '^IXIC': 'NASDAQ',
    '^RUT': 'Russell 2000',
    '^FTSE': 'FTSE 100',
    '^N225': 'Nikkei 225',
    '^NYA': 'NYSE'
}

df = df.rename(columns=column_mapping)
df.dropna()

returns_df = df.pct_change()*100
returns_df.columns = [col + '_return' for col in returns_df.columns]
returns_df = returns_df.dropna()

###

FI = ['VNINDEX','HNX','VN30','HNX30','UPCOM']
data = stock_historical_data('VNINDEX', '2012-03-19', '2023-09-05', "1D", "index")
d_new = data.set_index('time')
d_new['VNINDEX'] = data['close']
df1 = pd.DataFrame()
for symbol in FI:
    data = stock_historical_data(symbol, '2012-03-19', '2023-09-05', "1D", "index")
    data = data.set_index('time')
    df1[symbol] = data['close']
df1 = df1.dropna()

returns_df1 = df1.pct_change()*100
returns_df1.columns = [col + '_return' for col in returns_df1.columns]
returns_df1 = returns_df1.dropna()
returns_df1.head(42069)

###

lagreturn = pd.DataFrame()

for column in returns_df.columns:
    lagreturn[f'{column}_Lag1'] = returns_df[column].shift(1)
for column in returns_df.columns:
    lagreturn[f'{column}_Lag2'] = returns_df[column].shift(2)
lagreturn = lagreturn.dropna()

lagreturn1 = pd.DataFrame()
for column in returns_df1.columns:
    lagreturn1[f'{column}_Lag1'] = returns_df1[column].shift(1)
for column in returns_df1.columns:
    lagreturn1[f'{column}_Lag2'] = returns_df1[column].shift(2)
lagreturn1 = lagreturn1.dropna()

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


In [11]:
def do_tre_vnstock(symbol, start_date, end_date, lags=10): 
    ts = stock_historical_data(symbol,start_date, end_date)
    ts = ts.dropna(subset=['close'])


    tsret = pd.DataFrame(index=ts.index)
    tsret['time']=ts['time']
    tsret["Today"] = ts["close"].pct_change()*100

    for i,x in enumerate(tsret["Today"]):
        if (abs(x) < 0.0001):
            tsret["Today"][i] = 0.0001

    for i in range(0, lags):
        tsret["Lag%s" % str(i+1)] = tsret["Today"].shift(i+1)

    tsret["Direction"] = np.sign(tsret["Today"])
    tsret = tsret.dropna()
    tsret = tsret.set_index('time')
    bpt = pd.merge(tsret, lagreturn, left_index=True, right_index=True)
    bpt1 = pd.merge(bpt, lagreturn1, left_index=True, right_index=True)
    bpt1.fillna(0, inplace=True)
    bpt1 = bpt1[(bpt1 != 0).all(axis=1)]
    return bpt1

In [14]:
#MODEL
list_i = []
list_pcapercent = []
list_return_price = []
list_rmse, list_mse, list_mae =[], [], []
list_k = []
list_r2_train = []
list_r2_test = []

scaler = StandardScaler()
pca = PCA(n_components = 20)

for i in data_ticker['ticker']:
  try:
    data = do_tre_vnstock(i, '2012-03-19', '2023-09-05', lags=10)

    a = data.loc[:, ~data.columns.isin(['Today'])]

    a_scale = scaler.fit_transform(a)
    a_scale_df = pd.DataFrame(a_scale, columns=a.columns)
    pF = pca.fit_transform(a_scale_df)
    pF_df = pd.DataFrame(data = pF) 
    ss = pca.explained_variance_ratio_
    total = sum(ss)
    list_pcapercent.append(total)
    b = pd.DataFrame()
    b['Today'] = data['Today'].reset_index(drop=True)
    pF_final = pd.concat([pF_df, b], axis=1)
    
    X = pF_final.loc[:, ~pF_final.columns.isin(['Today'])]
    y = pF_final['Today']

    list_k1=[]
    list_score=[]
    for k in np.arange(0.1,0.5,0.01):
      model = LinearRegression()
      X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = k, random_state = 12, shuffle=False)
      model.fit(X_train, y_train)
      yhat_test = model.predict(X_test)
      score = model.score(X_test, y_test)
      list_k1.append(k)
      list_score.append(score)

    score_max = max(list_score)
    a = list_score.index(score_max)
    k = list_k1[a]

    model = LinearRegression()
    X_train, X_test,y_train, y_test=train_test_split(X,y,test_size=k,random_state=12, shuffle=False)
    model.fit(X_train,y_train)
    yhat_test = model.predict(X_test)
    r2_train = model.score(X_train, y_train)
    r2_test = model.score(X_test, y_test)
    mse = mean_squared_error(y_test, yhat_test, squared=True)
    rmse = mean_squared_error(y_test, yhat_test, squared=False)
    mae = mean_absolute_error(y_test, yhat_test)

    y = list(data["Today"])
    X_predict = [X.iloc[-1].values.tolist()]
    Y_predict = model.predict(X_predict)

    list_i.append(i)
    list_k.append(k)
    list_return_price.append(Y_predict)
    list_r2_train.append(r2_train)
    list_r2_test.append(r2_test)
    list_rmse.append(rmse)
    list_mse.append(mse)
    list_mae.append(mae)

    print('Stock:',i, f' Predict: {Y_predict.item():.2f}',f' R2_train: {r2_train:.2f}',f' R2_test: {r2_test:.2f}',
          f' RMSE: {rmse:.2f}',f'MSE: {mse:.2f}', f'MAE: {mae:.2f}',f' test size: {k:.2f}',f'pca: {total:.2f}')
  except Exception as e:
    print(f"Stock: {i} -Unpredictable!\n" f'{e}')
    continue

Stock: SSI  Predict: -2.20  R2_train: 0.56  R2_test: 0.56  RMSE: 1.87 MSE: 3.51 MAE: 1.50  test size: 0.33 pca: 0.91
Stock: BCM  Predict: -0.87  R2_train: 0.40  R2_test: 0.33  RMSE: 1.16 MSE: 1.34 MAE: 0.96  test size: 0.10 pca: 0.90
Stock: VHM  Predict: 1.48  R2_train: 0.50  R2_test: 0.54  RMSE: 1.39 MSE: 1.92 MAE: 1.03  test size: 0.14 pca: 0.90
Stock: VIC  Predict: 1.15  R2_train: 0.45  R2_test: 0.40  RMSE: 1.80 MSE: 3.23 MAE: 1.20  test size: 0.36 pca: 0.90
Stock: VRE  Predict: -1.56  R2_train: 0.52  R2_test: 0.49  RMSE: 1.75 MSE: 3.06 MAE: 1.25  test size: 0.33 pca: 0.91
Stock: BVH  Predict: 1.58  R2_train: 0.59  R2_test: 0.44  RMSE: 1.63 MSE: 2.65 MAE: 1.18  test size: 0.45 pca: 0.90
Stock: POW  Predict: 1.83  R2_train: 0.54  R2_test: 0.51  RMSE: 1.56 MSE: 2.42 MAE: 1.25  test size: 0.31 pca: 0.90
Stock: GAS  Predict: 1.74  R2_train: 0.53  R2_test: 0.49  RMSE: 1.39 MSE: 1.92 MAE: 1.00  test size: 0.41 pca: 0.90
Stock: ACB  Predict: 1.33  R2_train: 0.50  R2_test: 0.46  RMSE: 1.36 

In [15]:
KQ = pd.DataFrame({'Stock': list_i, 'Predict': list_return_price, 'R2_train': list_r2_train,'R2_test': list_r2_test,
                    'RMSE': list_mse,'MSE': list_rmse, 'MAE': list_mae,'test size': list_k, 'pca': list_pcapercent})
KQ.to_excel("Kết quả sau PCA.xlsx")
KQ.head(42069)

,Stock,Predict,R2_train,R2_test,RMSE,MSE,MAE,test size,pca
0,SSI,[-2.1964641358479535],0.559358,0.559503,3.514811,1.874783,1.499303,0.33,0.908270
1,BCM,[-0.8671243328133638],0.399426,0.334936,1.335158,1.155490,0.955836,0.10,0.900623
2,VHM,[1.4834444297620109],0.499441,0.537049,1.918825,1.385217,1.025881,0.14,0.904386
3,VIC,[1.1530264265879078],0.446722,0.398442,3.228988,1.796939,1.200836,0.36,0.902092
4,VRE,[-1.5630174225744167],0.522559,0.489124,3.056724,1.748349,1.248693,0.33,0.905858
...,...,...,...,...,...,...,...,...,...
389,TDP,[0.9820020142299866],0.326155,0.409577,4.967196,2.228721,1.944102,0.10,0.901230
390,NO1,[-0.023398252090538973],0.485162,0.383804,7.639444,2.763954,2.038791,0.12,0.909929
391,L10,[2.3828856098019573],0.611020,0.666165,9.944512,3.153492,2.592613,0.14,0.894500
392,REE,[1.3223607837990228],0.455368,0.440295,3.177998,1.782694,1.245959,0.42,0.900736
